In [1]:
using DevCommunities

┌ Warning: The plotting functionality of HyperNetX will not work!
│ Conda Python HyperNetX not found.
│ To test your installation try running `using PyCall;pyimport("networkx");pyimport("hypernetx")`
└ @ SimpleHypergraphs ~/.julia/packages/SimpleHypergraphs/DNs9a/src/SimpleHypergraphs.jl:62


┌ Warning: The plotting functionality of HyperNetX will not work!
│ Conda Python HyperNetX not found.
│ To test your installation try running `using PyCall;pyimport("networkx");pyimport("hypernetx")`
└ @ SimpleHypergraphs /Users/ddevin/.julia/packages/SimpleHypergraphs/DNs9a/src/SimpleHypergraphs.jl:62


Build all structures

In [2]:
hgs_reddit = build_all_hgs("reddit")
dfs_raw_reddit = build_raw_dfs("reddit")
dfs_processed_reddit = build_all_dfs("reddit")
hgs_reddit_labelled = build_all_hgs_labeled_reddit(dfs_processed_reddit, dfs_raw_reddit)
println("Construction of all structures done")

1


2


3


4
5


6


...

Building hypergraph 5


Building hypergraph 6


Building hypergraph 7


Building hypergraph 8


8-element Vector{Any}:
 Union{Nothing, Bool}[true nothing … nothing nothing; true nothing … nothing nothing; … ; nothing nothing … nothing nothing; nothing nothing … true nothing]
 Union{Nothing, Bool}[true true … nothing nothing; true nothing … nothing nothing; … ; nothing nothing … nothing nothing; nothing nothing … nothing true]
 Union{Nothing, Bool}[true nothing … nothing nothing; true nothing … nothing nothing; … ; nothing nothing … nothing nothing; nothing nothing … nothing nothing]
 Union{Nothing, Bool}[true nothing … nothing nothing; true nothing … nothing nothing; … ; nothing nothing … nothing nothing; nothing nothing … nothing nothing]
 Union{Nothing, Bool}[true nothing … nothing nothing; true nothing … nothing nothing; … ; nothing nothing … true nothing; nothing nothing … nothing true]
 Union{Nothing, Bool}[true nothing … nothing nothing; true true … nothing nothing; … ; nothing nothing … nothing nothing; nothing nothing … nothing true]
 Union{Nothing, Bool}[true nothing … n

Community detection

In [6]:
my_tags_r = ["rust" "elixir" "Clojure" "typescript" "Julia" "Python" "delphi" "golang" "SQL" "csharp" "Kotlin" "swift" "dartlang" "HTML" "solidity" "javascript" "fsharp" "bash" "lisp" "apljk"]

cds_reddit = community_detection(hgs_reddit_labelled)

temporal_comms_r, percentages_r = temporal_communities(cds_reddit, 0.05)
all_communities_aggregated_r = t_communities_aggregation_reddit(temporal_comms_r, percentages_r, my_tags_r, dfs_processed_reddit)


Building communities of hg 1


Building communities of hg 2


Building communities of hg 3


Building communities of hg 4


Building communities of hg 5


Building communities of hg 6


Building communities of hg 7


Building communities of hg 8


8-element Vector{Any}:
 Dict{Any, Any}("dartlang" => Set(Any[11360, 11130, 11348, 11284, 11273, 11123, 11328, 11349, 11198, 11326  …  11327, 11347, 11280, 11325, 11350, 11136, 11337, 11213, 11323, 11257]), "SQL" => Set(Any[35463, 35129, 7144, 23896, 35387, 17486, 22700, 35478, 35472, 7559  …  35239, 35482, 35109, 35035, 5137, 12390, 35480, 10808, 31372, 2315]), "csharp" => Set(Any[29965, 29454, 30438, 30270, 30960, 29998, 29914, 27167, 28804, 29776  …  29711, 27944, 28914, 30542, 25304, 28587, 31139, 29663, 30145, 29935]), "javascript" => Set(Any[34071, 32167, 33084, 33236, 31837, 15610, 32425, 32353, 32220, 33075  …  34229, 33012, 33779, 34086, 33358, 15706, 32522, 32372, 32792, 33347]), "golang" => Set(Any[13311, 31594, 13057, 12778, 14999, 12560, 15002, 12554, 13982, 13363  …  12558, 12571, 15042, 12709, 12783, 15271, 33881, 14073, 15761, 15432]), "Julia" => Set(Any[31426, 31503, 31636, 31659, 31651, 31663, 31549, 24377, 31678, 4934  …  31662, 31457, 31552, 31452, 31429, 4754, 31376

In [7]:
for trim in 1:7
    write(io, "intersection\n")
    for (k, v) in all_communities_aggregated[trim]
        if haskey(all_communities_aggregated[trim+1], k)
            pre_comm = all_communities_aggregated[trim][k]
            post_comm = all_communities_aggregated[trim+1][k]
            pre_comm = [get_vertex_meta(hgs[trim], elem) for elem in pre_comm]
            post_comm = [get_vertex_meta(hgs[trim+1], elem) for elem in post_comm]
            pre_comm = unique(pre_comm)
            post_comm = unique(post_comm)
            intersezione = intersect(pre_comm, post_comm)
            perc = length(intersezione) / length(pre_comm)
            #  TAG || LENGTH(PRE_COMM) || LENGTH(POST_COMM) || LENGTH(INTERSECTION) || PERC
            write(io, k)
            write(io, ",")
            write(io, string(length(pre_comm)))
            write(io, ",")
            write(io, string(length(post_comm)))
            write(io, ",")
            write(io, string(length(intersezione)))
            write(io, ",")
            write(io, string(perc))
            write(io, "\n")
        end
    end
    write(io, "-------\n\n")
    write(io, "Migration \n")
    # find how many user move from each community to another
    for (k, v) in all_communities_aggregated[trim]
        if haskey(all_communities_aggregated[trim+1], k)
            pre_comm = all_communities_aggregated[trim][k]
            post_comm = all_communities_aggregated[trim+1][k]
            pre_comm = [get_vertex_meta(hgs[trim], elem) for elem in pre_comm]
            post_comm = [get_vertex_meta(hgs[trim+1], elem) for elem in post_comm]
            pre_comm = unique(pre_comm)
            post_comm = unique(post_comm)
            possible_moved = setdiff(pre_comm, post_comm)
            for (k2, v2) in all_communities_aggregated[trim+1]
                pp_comm = all_communities_aggregated[trim+1][k2]
                pp_comm = [get_vertex_meta(hgs[trim+1], elem) for elem in pp_comm]
                intersezione = intersect(possible_moved, pp_comm)
                if length(intersezione) > 0
                    # println("MOving from ", k, " ", length(possible_moved), " to ", k2, " ", length(all_communities_aggregated[trim+1][k2]), " Intersezione ", length(intersezione))
                    write(io, k)
                    write(io, ",")
                    write(io, string(length(possible_moved)))
                    write(io, ",")
                    write(io, k2)
                    write(io, ",")
                    write(io, string(length(pp_comm)))
                    write(io, ",")
                    write(io, string(length(intersezione)))
                    write(io, "\n")
                end
            end
        end
        write(io, "*****\n")
    end

    # user that disappeared from the community
    possible_ghost = Set()
    set_user1 = Set()
    set_user2 = Set()
    for (k, v) in all_communities_aggregated[trim]
        pre_comm = all_communities_aggregated[trim][k]
        pre_comm = [get_vertex_meta(hgs[trim], elem) for elem in pre_comm]
        set_user1 = union(set_user1, pre_comm)
    end
    for (k2, v2) in all_communities_aggregated[trim+1]
        post_comm = all_communities_aggregated[trim+1][k2]
        post_comm = [get_vertex_meta(hgs[trim+1], elem) for elem in post_comm]
        set_user2 = union(set_user2, post_comm)  
    end
    possible_ghost = setdiff(set_user1, set_user2)
    # if length(possible_ghost) > 0
        # println("User disappeared ", length(possible_ghost))
        write(io, "User disappeared ")
        write(io, string(length(possible_ghost)))
        write(io, "\n")
    # end

    new_users = setdiff(set_user2, set_user1)
    # if length(new_users) > 0
        # println("New users ", length(new_users))
        write(io, "New users ")
        write(io, string(length(new_users)))
        write(io, "\n")
    # end
end

UndefVarError: UndefVarError: io not defined

Quantiles

In [ ]:
for i in 1:8
    size_communities = length.(cds_reddit[i].np)
    v = quantile(size_communities, [0.25, 0.5, 0.75, 0.9])
    display("trim " * string(i))
    for j in 1:4
        display(v[j] * " ")
    end
end